## Árvore AVL


O código abaixo ilustra uma árvore AVL. Uma árvore AVL é uma árvore binária balanceada. O balanceamento é realizado após cada inclusão ou exclusão de nós.

exclusãoDado 1 nó qualquer da árvore a diferença entre a altura da sub-árvore da esquerda e da sub-árvore da direita será sempre menor ou igual a 1. Esta diferença é chamada de **fator de balanceamento** do nó.

In [1]:
#include <iostream>
#include <stdlib.h> 
using namespace std;

Estrutura para uma árvore AVL.

In [2]:
struct tNo {
    int chave; 
    tNo *esq, *dir, *pai;
    int fb; //fator de balanceamento (altura esquerda - altura direita)
};

Função para inicialização do nó, com alocação de memória e atribuição de valores para os componentes do nó.

In [3]:
tNo *criaNo (int chave){
    tNo *no = (tNo *)malloc (sizeof (tNo));
    no->chave = chave;
    no->esq = NULL;
    no->dir = NULL;
    no->pai = NULL; 
    no->fb = 0;
    return no;
}

Rotação à esquerda do nó, retorna o novo nó. Tem ponteiro para o pai.

In [4]:
tNo *rotEsquerda(tNo *p){
    tNo *q = p->dir;
    p->dir = q->esq;
    q->pai = p->pai;
    p->pai = q;
    if (q->esq != NULL)
        q->esq->pai = p;
    q->esq = p;
    return q;
}

Rotação à direita do nó, retorna o novo nó. Tem ponteiro para o pai.

In [5]:
tNo *rotDireita(tNo *p){
    tNo *q = p->esq;
    p->esq = q->dir;
    q->pai = p->pai;
    p->pai = q;
    if (q->dir != NULL)
        q->dir->pai = p;
    q->dir = p;
    return q;
}

Altura da árvore

In [6]:
int altura (tNo *p) {
    int he, hd;
    if (p == NULL) return -1;
    he = altura (p->esq);
    hd = altura (p->dir);
    if (he > hd)
        return he+1;
    else
        return hd+1;    
}

Função para ajustar ponteiro _pai_ da árvore binária, usada na função de exclusão.

In [7]:
void ajustaNoPai(tNo *no, tNo *novo){
    if (no->pai != NULL) {
        if (no->pai->esq == no)
            no->pai->esq = novo;
        else
            no->pai->dir = novo;
        if (novo != NULL)
           novo->pai = no->pai;
    }
}

Função que ajusta um nó desbalanceado em uma árvore AVL. Um nó está desbalanceado quando a diferença de altura entre a subárvore da direita e da esquerda do nó é maior que 1.

Caso o fator de balanceamento for -2, está desbalanceada para a esquerda. Caso for 2, está desbalanceada para a direita.

A função tem 4 casos possíveis:
- esq-esq : 1 rotação à direita do nó
- esq-dir : 1 rotação à esquerda do nó à esquerda e 1 rotação à direita do nó
- dir-dir : 1 rotação à esquerda do nó
- dir-esq : 1 rotação à direita do nó à direita e 1 rotação à esquerda do nó

In [8]:
tNo* ajustaAVL(tNo * no, int *calcula_fb){
    cout << "\najusta "<< no->chave <<" \n";
    tNo *noAjuste;//, *pai = no->pai;
    if (no->fb == -2) { //desbalanceado na  esquerda
        if (no->esq != NULL && no->esq->fb > 0) {
            cout << "esq." << no->esq->chave;
            no->esq = rotEsquerda(no->esq);
        }
        cout << " dir." << no->chave;    
        noAjuste = rotDireita(no);        
    } else {        
        if (no->dir->fb < 0){
            cout << "dir."<< no->dir->chave;
            no->dir = rotDireita(no->dir);
        }
        cout << " esq." << no->chave;
        noAjuste = rotEsquerda(no);
        
    }
    if (noAjuste->pai != NULL){
        if (noAjuste->pai->esq == no)
            noAjuste->pai->esq = noAjuste;
        else
            noAjuste->pai->dir = noAjuste;  
    }
    noAjuste->fb = 0;
    noAjuste->esq->fb = (altura(noAjuste->esq->dir) - altura(noAjuste->esq->esq));
    noAjuste->dir->fb = (altura(noAjuste->dir->dir) - altura(noAjuste->dir->esq));
    *calcula_fb = 0;
    return noAjuste;
}

Funcão para inclusão de novo nó na árvore AVL, dado um nó raiz.

Possui um parâmetro extra, chamado **calcula_fb**, para indicar se uma determinada sub-árvore teve sua altura alterada.

In [9]:
tNo* inclui (tNo *no, int c, int *calcula_fb){
    if (no == NULL) {
        no = criaNo(c);        
        (*calcula_fb) = 1;
        return no;    
    }
    if (c < no->chave){
        no->esq = inclui(no->esq, c, calcula_fb);
        no->esq->pai = no;
        if (*calcula_fb) no->fb --;
    }
    else {
        no->dir = inclui(no->dir, c, calcula_fb);
        no->dir->pai = no;
        if (*calcula_fb) no->fb ++;
    }
    if (*calcula_fb){
        if (no->fb == 0) *calcula_fb = 0;
        if (no->fb == 2 || no->fb == -2) // se está desbalanceada, precisa chamar a função de ajuste
           no = ajustaAVL(no, calcula_fb);
    }
    return no;
}

Função que lê um token, separado por espaço, e converte para um número inteiro.

In [10]:
int token_to_num(const char *str, int *indice){    
    char token[100];
    int i = 0;
    while (str[*indice] != '\0' && str[*indice] != ' '){
        token[i] = str[*indice];
        i++;
        (*indice)++;
    }
    token[i] = '\0';
    (*indice)++;    
    return atoi(token);
}

Função que monta uma árvore binária recebendo como entrada uma árvore com parênteses aninhados. Não há suporte a erros de entrada, por isso a árvore passada como parâmetro deve estar correta.

In [11]:
tNo* montaarvore(const char *str){
    tNo *raiz = NULL;
    int i = 0, v =0, calcula_fb = 0;
    raiz = inclui(NULL, token_to_num(str,&i), &calcula_fb);
    while (str[i]!='\0'){
        raiz = inclui (raiz, token_to_num(str, &i), &calcula_fb);                
    }
    return raiz;        
}

Operação de visita do nó. Neste caso é uma impressão simples.

In [12]:
void visita(tNo * no){
    cout << no->chave << ':'<< no->fb << '.';
}

Percurso da árvore em **PRÉ-ORDEM**.

In [13]:
void preordem(tNo *no){
    cout << "(";
    if (no != NULL){
        visita(no);
        preordem(no->esq);
        preordem(no->dir);
    }
    cout << ")";
}

Percurso da árvore em **ORDEM**.

In [14]:
void emordem(tNo *no){    
    if (no !=NULL){
        emordem(no->esq);
        visita(no);
        emordem(no->dir);
    }
}

Percurso da árvore em **PÓS-ORDEM**.

In [15]:
void posordem(tNo * no){
    if (no !=NULL){    
        posordem(no->esq);        
        posordem(no->dir);
        visita(no);
    }
}

Função que retorna o nó mínimo de uma sub-árvore. O nó mínimo de uma subárvore é o nó com a menor chave.

In [16]:
tNo *min(tNo *no){
    if (no->esq == NULL) return no;
    else
        return min(no->esq);
}

Função que retorna o nó sucessor de um determinado nó. A chave do nó deverá ser o valor imediatamente superior, em ordem crescente. 

As funções do antecessor e máximo possuem ideia semelhante as do sucessor e mínimo, porém para o lado contrário da árvore.

In [17]:
tNo *sucessor (tNo *no){
    tNo *s = NULL;
    if (no->dir != NULL) return min (no->dir);
    else {
        s = no->pai;
        while (s != NULL && no == s->dir) {
            no = s;
            s = s->pai;
        }        
    }
    return s;
}

Busca em árvore binária.

In [18]:
tNo *busca(tNo *no, int chave){
    if (no == NULL) return NULL;
    if (no->chave == chave) return no;
    if (chave < no->chave)
        return busca (no->esq, chave);
    else
        return busca (no->dir, chave);
}

Ajusta fator de balanceamento do pai.

In [19]:
tNo *ajustaFBPai(tNo *p){
    tNo *no = p;
    while (no->pai != NULL && no->fb != -2 && no->fb != 2){
        if (no->pai->esq == no)
            no->pai->fb++;
        else
            no->pai->fb--;
        no = no->pai;
    }
    return no;
}

Conta o total de nós.

In [20]:
int contaNos(tNo *no){
    if (no!=NULL) {
        return contaNos(no->esq) + contaNos(no->dir) + 1;
    } else
        return 0;
}

Função que exclui um nó da árvore AVL. Usa a regra do sucessor. Retorna a raiz da árvore, pois a árvore poderá ter uma nova raiz, caso seja o nó a ser excluído.

Chama funções de ajuste do fator de balanceamento do nó e dos nós pais.

A mesma função poderia ser adaptada para usar a regra do antecessor.

In [21]:
tNo* exclui (tNo *no, tNo *raiz) {
    int calcula_fb;
    tNo *direitaSuc, *s, *novaRaiz = raiz;
    if (no == NULL) return NULL;    
    tNo *noAjuste = no->pai;
    if (no->esq == NULL && no->dir == NULL) { //nó folha
        if (no == raiz) {//exclusao da raiz em árvore com 1 nó
            free(no);
            return NULL;
        }
        if (no->pai->esq == no)
            no->pai->fb++;
        else
            no->pai->fb--;
        noAjuste = ajustaFBPai(no->pai);               
        ajustaNoPai(no,NULL);
        free (no);        
    } else {
        if (no->esq == NULL){ // nó com 1 filho direito
            noAjuste = ajustaFBPai(no);
            ajustaNoPai(no, no->dir);
            if (no->pai == NULL) novaRaiz = no->dir;
            free (no);
        } else {
            if (no->dir == NULL){ //nó com 1 filho esquerdo
                noAjuste = ajustaFBPai(no);
                ajustaNoPai(no, no->esq);
                if (no->pai == NULL) novaRaiz = no->esq;
                free(no);
            }
            else {            //nó com 2 filhos
                int ultimoSucessor;
                s = sucessor (no);
                direitaSuc = s->dir;
                ultimoSucessor = s->esq == NULL && s->dir == NULL;
                ajustaNoPai(s, s->dir);
                s->esq = no->esq;
                s->dir = no->dir;
                s->fb = no->fb;
                ajustaNoPai(no, s);   
                s->esq->pai = s;                       
                if (ultimoSucessor) {//ultimo sucessor
                    s->fb --;
                    noAjuste = s;
                } else
                    s->dir->pai = s;
                if (no == raiz) {
                    novaRaiz = s;
                    s->pai = NULL; //verificar essa linha. já deveria ser NULL
                }
                free(no);
                if (s->fb != -2 && direitaSuc != NULL) //verifica se precisa ajustar 'pra cima'
                    noAjuste = ajustaFBPai(direitaSuc);
            }
        }
    } 
    while (noAjuste != NULL && (noAjuste->fb == 2 || noAjuste->fb == -2)){ 
        // se está desbalanceada após a exclusão,
        calcula_fb = 1;     //chama a função de ajuste
        no = ajustaAVL(noAjuste, &calcula_fb);
        if (no->pai == NULL){ //para se o nó ajustado for o nó raiz
            novaRaiz = no;
            noAjuste = NULL;
        }
        else
            noAjuste = ajustaFBPai(noAjuste->pai);
    }
    return novaRaiz;
}

In [22]:
void iniciaprograma(){
    tNo *raiz = NULL, *no= NULL;
    int v;
    char entrada[] = "50 30 90 20 40 100 35 45 85 88 21 37\0";
    //char entrada[] = "50 90 100\0"; //árvore mais simples
    
    raiz=montaarvore(entrada);    
    cout << "\nPercurso em pré-ordem: ";
    preordem(raiz); cout << "\n";
    cout << "total de nós: " << contaNos(raiz);
    cout << "\naltura: " << altura(raiz);

    //nós com casos complexos para testar com a árvore inteira : 100, 88, 21, 50, 90
    v = 37; //testa exclusão do sucessor
    cout << "\n\nexclusão :" << v << "\n";
    raiz = exclui( busca(raiz,v), raiz); cout << "\n";
    preordem(raiz); cout << "\n";
    cout << "total de nós: " << contaNos(raiz);      
    cout << "\naltura: " << altura(raiz);
}

In [23]:
iniciaprograma();


Percurso em pré-ordem: (50:-1.(30:1.(20:1.()(21:0.()()))(40:-1.(35:1.()(37:0.()()))(45:0.()())))(90:-1.(85:1.()(88:0.()()))(100:0.()())))
total de nós: 12
altura: 4

exclusão :37

(50:0.(30:0.(20:1.()(21:0.()()))(40:0.(35:0.()())(45:0.()())))(90:-1.(85:1.()(88:0.()()))(100:0.()())))
total de nós: 11
altura: 3